# Decision Trees

Let's implement a decision tree

We'll need two kinds of nodes:

1.  **Internal nodes** to  represent decisions
2.  **Leaf nodes** to represent classes

In [394]:
class Node(object):
    def __init__(self, children):
        self.children = children

class Internal(Node):
    def __init__(self, attrfn, children):
        print("Building internal node with predicate {0}".format(attrfn))        
        self.predicate = attrfn
        super().__init__(children)

class Leaf(Node):
    def __init__(self, label):
        print("Building leaf with class {0}".format(label))
        self.class_label = label
        super().__init__(None)

The basic algorithm is simple:

<code>
 build_tree(samples)
  if (y = 0 for all (x, y) in samples) return new leaf(0)
  else if (y = 1 for all (x,y) in samples) return new leaf(1)
  else
    chose best attribute x<sub>j</sub>
    s0 = all (x, y) in samples with x<sub>j</sub> = 0
    s1 = all (x, y) in samples with x<sub>j</sub> = 1
    return new node(x<sub>j</sub>, build_tree(s0), build_tree(s1))
</code>

In [395]:
from collections import Counter

def build_tree(samples, split, attrfns, classfn):
    """Build a decision tree
       Parameters:
       samples   -- list of samples, where each sample is a list of attributes
       split     -- function that takes a list of samples, a list of attribute functions, and 
                    a classfn function. The function splits the data based on the best attribute,
                    and returns a tuple:
                    1) dict of data split, where key is the attribute value, and the value
                       is a list of examples that have that attribute value
                    2) the attribute function used for this split
                    
                    Example:
                    
                    samples = [['Monday', 'Yellow', 'X'], ['Monday', 'Red', 'O']]
                    def day(s): return s[0]
                    def color(s): return s[1]
                    def classfn(s): return s[2]
                    
                    
                    split(samples, [day, color], classfn)  (may return) =>
                    
                    {'Yellow' => ['Monday', 'Yellow', 'X']
                     'Red'    => ['Monday', 'Red', 'O']} ,
                     color
                     
                     if this function determines there is no best way to split these samples,
                     it will return None, None
                      
       attrfns   -- list of attribute functions. Each of these functions can be applied to a 
                    a sample to get a specific attribute.
                    e.g., [day_attr, color_attr],  day_attr(sample) => "Monday"
                    or color_attr(sample) => "Red"
       classfn     -- function that takes a single sample and returns the class for that sample
                    e.g., classfn(sample) => 'X'
    """        
    if all(classfn(samples[0]) == classfn(sample) for sample in samples):
        print("Building leaf with {0} same-class samples".format(len(samples)))
        return Leaf(classfn(samples[0]))

    if not attrfns:
        print("Building leaf with {0} samples".format(len(samples)))
        return Leaf(Counter(classfn(sample) for sample in samples).most_common()[0][0])
    
    splits, attrfn = split(samples, attrfns, classfn)
    if not splits:
        print("Building leaf with {0} samples".format(len(samples)))
        return Leaf(Counter(classfn(sample) for sample in samples).most_common()[0][0])
    
    remaining_attrfns = [fn for fn in attrfns if fn != attrfn]

    child_nodes = dict()
    for key, group in splits.items():
        child_nodes[key] = build_tree(group, split, remaining_attrfns, classfn)
    return Internal(attrfn, child_nodes)

We have not defined split, that depends on the data and which criteria to use.

That brings us to entropy and information gain.


Let's take a quick detour and build entropy and information gain. We'll use information gain to pick an attribute (and measure how well it splits the data)

### Entropy and Information Gain

In [396]:
import math

def entropy(examples):
    '''
    Computes entropy of samples
    
    The min entropy is 0.0, the max entropy is log2(n), where n is number of classes.
    
    Parameters
    examples -- list of number of examples per class
    '''
    total = sum(examples)
    entropy = 0.0
    
    # filter out classes with 0 examples to compute - p * log(p) 
    # (i.e., we define 0 * log(0) == 0)
    for n in filter(None, examples):
        entropy -= (n/total) * math.log(n/total , 2)
    return entropy

We use entropy to measure the "purity" of a node. A node is 100% pure (entropy == 0.0) if it contains examples of the same class.

When a node is 100% impure, we'll get log2(n), where n is the number of classes.

In [397]:
0 == entropy([10]) # 100% pure, all same class

True

In [398]:
1 == entropy([5, 5]) # Max impurity, two classes, data is split across classes equally

True

In [399]:
entropy([5, 10])  # impure

0.9182958340544896

In [400]:
entropy([9, 1])  # 9 examples of 1 class, 1 example of another

0.4689955935892812

In [401]:
entropy([50, 1]) # purer split

0.1392329990550989

In [402]:
entropy([99, 1])

0.08079313589591118

In [403]:
entropy([10000, 1])

0.0014729006652121114

In [404]:
entropy([10**6, 1]) # almost pure

2.137424295738942e-05

In [405]:
print(math.log2(6))
math.log2(6) == entropy([1,1,1,1,1,1])  # Max impurity, 6 classes


2.584962500721156


True

#### Gain

In [406]:
def gain(examples, classfn, classes, attrfn, attrvals):
    '''
    Calculates information gain after splitting on an attribute
    
    Parameters
    examples - list of examples. Each example has attributes.
    class_fn - function that returns the class of an example.
    classes  - list of all classses
    attrfn   - function that returns the value of a specific attribute given an example.
       e.g., attrnfn(example) -> attribute value
    attrvals - list of all possible values for the attribute used for this split
       e.g., [1, 2, 3], or ['red', 'yellow', 'green']
    
    '''

    en = entropy(counts_per_class(examples, classfn, classes))    
    total = len(examples)

    for val in attrvals:
        # Get all examples whose value for the attribute is val
        sv = list(filter(lambda example: attrfn(example) == val, examples))
        en -= len(sv)/total * entropy(counts_per_class(sv, classfn, classes))
    return en

In [407]:
moon_day_examples = [
    ['clear', 'cold', 'winter', 'moon'],
    ['rainy', 'warm', 'spring', 'no-moon'],
    ['cloudy', 'cold', 'winter', 'no-moon'],
    ['clear', 'warm', 'summer', 'moon'],
    ['rainy', 'cold', 'fall', 'no-moon'],
    ['rainy', 'cold', 'spring', 'no-moon'],
    ['clear', 'cold', 'spring', 'moon'],
]

In [408]:
def moon(example):
    return example[3]

In [409]:
def counts_per_class(examples, classfn, classes):
    '''
     Returns the distribution of classes for a subset of examples.
       e.g., classfn(examples) -> [3, 4, 5]  (3 are of class 0, 4 of class 1, 5 of class 2)
    '''
    dist = dict()
    for cls in classes:
        dist[cls] = 0
    
    for example in examples:
        cls = classfn(example)
        dist[cls] += 1
    
    flat_dist = []
    for cls in classes:
        flat_dist.append(dist[cls])
    return flat_dist

In [410]:
counts_per_class(moon_day_examples, moon, ['moon', 'no-moon'])  # 3 'moon' and 4 'non-moon'

[3, 4]

In [411]:
def sky_condition(example): return example[0]
def temp(example): return example[1]
def season(example): return example[2]

In [412]:
def attrvalues(examples, attrfn):
    return set(attrfn(example) for example in examples)

In [413]:
classes = ['moon', 'no-moon']

In [414]:
sky = attrvalues(moon_day_examples, sky_condition)
sky

{'clear', 'cloudy', 'rainy'}

What's the gain if we split by **sky condition**?

In [415]:
gain(moon_day_examples, moon, classes, sky_condition, sky)

0.9852281360342516

What's the gain if we split by **temperature**?

In [416]:
temp_values = attrvalues(moon_day_examples, temp)
gain(moon_day_examples, moon, classes, temp, temp_values)

0.005977711423774124

What's the gain if we split by **season**?

In [417]:
season_values = attrvalues(moon_day_examples, season)
gain(moon_day_examples, moon, classes, season, season_values)

0.3059584928680419

As expected, sky condition is the best split to figure out which day we'll see the moon

Another example: when to play tennis?

In [418]:
tennis_dataset = [line.split() for line in """
sunny    hot  high   weak   no
sunny    hot  high   strong no
overcast hot  high   weak   yes
rain     mild high   weak   yes
rain     cool normal weak   yes
rain     cool normal strong no
overcast cool normal strong yes
sunny    mild high   weak   no
sunny    cool normal weak   yes
rain     mild normal weak   yes
sunny    mild normal strong yes
overcast mild high   strong yes
overcast hot  normal weak   yes
rain     mild high   strong no
""".split('\n')[1:-1]]

In [419]:
tennis_dataset

[['sunny', 'hot', 'high', 'weak', 'no'],
 ['sunny', 'hot', 'high', 'strong', 'no'],
 ['overcast', 'hot', 'high', 'weak', 'yes'],
 ['rain', 'mild', 'high', 'weak', 'yes'],
 ['rain', 'cool', 'normal', 'weak', 'yes'],
 ['rain', 'cool', 'normal', 'strong', 'no'],
 ['overcast', 'cool', 'normal', 'strong', 'yes'],
 ['sunny', 'mild', 'high', 'weak', 'no'],
 ['sunny', 'cool', 'normal', 'weak', 'yes'],
 ['rain', 'mild', 'normal', 'weak', 'yes'],
 ['sunny', 'mild', 'normal', 'strong', 'yes'],
 ['overcast', 'mild', 'high', 'strong', 'yes'],
 ['overcast', 'hot', 'normal', 'weak', 'yes'],
 ['rain', 'mild', 'high', 'strong', 'no']]

In [420]:
def outlook(x): return x[0]
def temperature(x): return x[1]
def humidity(x): return x[2]
def wind(x): return x[3]

def play_tennis(x): return x[4]

In [421]:
outlook_values = attrvalues(tennis_dataset, outlook)
gain(tennis_dataset, play_tennis, ['no', 'yes'], outlook, outlook_values)

0.2467498197744391

In [422]:
humidity_values = attrvalues(tennis_dataset, humidity)
gain(tennis_dataset, play_tennis, ['no', 'yes'], humidity, humidity_values)

0.15183550136234136

In [423]:
wind_values = attrvalues(tennis_dataset, wind)
gain(tennis_dataset, play_tennis, ['no', 'yes'], wind, wind_values)

0.04812703040826927

In [424]:
temperature_values = attrvalues(tennis_dataset, temperature)
gain(tennis_dataset, play_tennis, ['no', 'yes'], temperature, temperature_values)

0.029222565658954647

Outlook is the best way to split our data first!

Let's use an interesting dataset

### Credit Card Application Dataset

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

In [425]:
import pandas as pd

In [426]:
cc_data = pd.read_csv('dataset/crx.data', header=None)

In [427]:
cc_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


Column 15 is the class. 

"+" means the credit card application was approved

"-" means it was denied


In [428]:
cc_data.shape

(690, 16)

In [429]:
import numpy as np
shuffled_data = cc_data.sample(frac=1).reset_index(drop=True)
shuffled_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,a,38.33,4.415,u,g,c,v,0.125,f,f,0,f,g,00160,0,-
1,b,22.92,3.165,y,p,c,v,0.165,f,f,0,f,g,00160,1058,-
2,b,49.17,2.290,u,g,ff,ff,0.290,f,f,0,f,g,00200,3,-
3,b,22.25,0.460,u,g,k,v,0.125,f,f,0,t,g,00280,55,-
4,b,50.75,0.585,u,g,ff,ff,0.000,f,f,0,f,g,00145,0,-


In [430]:
def cc_split(samples, attrfns, classfn):
    # Compute information gain for every attfn used
    gains = []
    for attrfn in attrfns:
        g = gain(samples, classfn, ['+', '-'], attrfn, attrvalues(samples, attrfn))
        gains.append((g, attrfn))
    
    g, fn = max(gains, key=lambda x: x[0])
    print("max gain is {0}: {1}".format(fn.__name__, g))     
    groups = group_by_fn(samples, fn)
    for key, val in groups.items():
        print("{0}: {1}".format(key, len(val)))
    return groups, fn

In [431]:
def group_by_fn(samples, fn):
    vals = attrvalues(samples, fn)
    groups = dict()

    for val in vals:
        groups[val] = []

    for x in samples:
        val = fn(x)
        groups[val].append(x)

    return groups    

In [432]:
def cc_class(x): return x[15]

In [433]:

# Let's build some attribute functions for categorical data

def zeroth(x): return x[0]
def third(x): return x[3]
def fourth(x): return x[4]
def fifth(x): return x[5]
def sixth(x): return x[6]
def eight(x): return x[8]
def ninth(x): return x[9]
def eleventh(x): return x[11]
def twelveth(x): return x[12]

cc_att_fns = [zeroth, third, fourth,
#               fifth,
              sixth, eight, ninth, eleventh, twelveth]

In [434]:
groups, fn = cc_split(shuffled_data[:100].values.tolist(), cc_att_fns, cc_class)

max gain is eight: 0.4996268261304999
t: 47
f: 53


The eigth attribute gave us the best gain.


Here's how this data was split:

In [435]:
for key, examples in groups.items():
    print("{0}: {1}".format(key, len(examples)))


t: 47
f: 53


In [436]:
x = shuffled_data[:100].values.tolist()

root = build_tree(x, cc_split, cc_att_fns, cc_class)

max gain is eight: 0.4996268261304999
t: 47
f: 53
max gain is sixth: 0.164936673715314
v: 29
z: 1
h: 12
bb: 4
?: 1
max gain is ninth: 0.0660138650613386
f: 11
t: 18
max gain is third: 0.04417739186726144
y: 1
u: 10
Building leaf with 1 same-class samples
Building leaf with class +
max gain is eleventh: 0.034851554559677256
t: 5
f: 5
max gain is zeroth: 0.01997309402197489
b: 3
a: 2
max gain is fourth: 0.0
g: 3
max gain is twelveth: 0.0
g: 3
Building leaf with 3 samples
Building leaf with class +
Building internal node with predicate <function twelveth at 0x116ce0268>
Building internal node with predicate <function fourth at 0x116cc41e0>
max gain is fourth: 0.0
g: 2
max gain is twelveth: 0.0
g: 2
Building leaf with 2 samples
Building leaf with class -
Building internal node with predicate <function twelveth at 0x116ce0268>
Building internal node with predicate <function fourth at 0x116cc41e0>
Building internal node with predicate <function zeroth at 0x116b58a60>
max gain is zeroth: 0.07

In [441]:
def count_nodes(node):
    if not node:
        return 0

    count = 0
    if node.children:
        for child in node.children.values():
            count += count_nodes(child)
    return count + 1

In [442]:
count_nodes(root)

50

### Using Chi-Squared statistic to prevent overfitting


|         | Passed           | Failed  | Total |
| ------------- |-------------- | ----- | ----- |
| Attended      | 25 (18.94)| 6 (12.06) |  31 |
| Skipped      |  8 (14.05) | 15 (8.94)  | 23 |
|Total | 33 | 21  | 54 |

Expected values are in parenthesis. They are computed using this formula:  (RowTotal*ColTotal)/GridTotal


In [443]:

def chi_sqrd(groups):
    '''Computes chi-squared statistic
       parameters:
       groups - list of tuples, where each tuple x,y  represents counts per class
    '''
    row_totals = []
    col_totals = [0.0, 0.0]
    
    for group in groups:
        col_totals[0] = col_totals[0] + group[0]
        col_totals[1] = col_totals[1] + group[1]
        row_totals.append(sum(group))

    grid_total = sum(col_totals)
    
    expected_values = []
    for i, group in enumerate(groups):
        a = (row_totals[i] * col_totals[0]) / grid_total
        b = (row_totals[i] * col_totals[1]) / grid_total
        expected_values.append((a, b))
    
    chi_sqrd_stat = 0.0
    for observed, expected in zip(groups, expected_values):
        chi_sqrd_stat += (observed[0] - expected[0]) * (observed[0] - expected[0]) / expected[0]
        chi_sqrd_stat += (observed[1] - expected[1]) * (observed[1] - expected[1]) / expected[1]

    return chi_sqrd_stat

In [444]:
chi_sqrd([(25, 6), (8, 15)]) # should be ~ 11.68

11.686016648148486

In [445]:
def chi_sqrd_from_groups(a, b, classfn, groups):
    ''' computes chi squared statistic after splitting using attribute function attrnf
    
        parameters:
        a       - first class
        b       - second class
        classfn - function that takes a single sample and returns the class of that sample
        groups  - sequence of groups, where each group is a list of samples 
    '''
    # For each group, compute the number of samples per class
    group_counts = []
    for group in groups:
        counts = [0, 0]
        for sample in group:
            if classfn(sample) == a:
                counts[0] += 1
            else:
                counts[1] += 1
        group_counts.append(tuple(counts))
    print("groups in chi_sqrd_from_groups: ", group_counts)
    return chi_sqrd(group_counts)   

We can revise our split function to use the chi square statistics

In [446]:
from scipy import stats

def cc_split_chi(samples, attrfns, classfn):
    # Compute information gain for every attfn used
    gains = []
    for attrfn in attrfns:
        g = gain(samples, classfn, ['+', '-'], attrfn, attrvalues(samples, attrfn))
        gains.append((g, attrfn))
    
    
    splits = sorted(gains, key=lambda x: x[0])
    g, fn, p, best_groups = None, None, None, None
    for gn, f in splits:
        groups = group_by_fn(samples, f)
        chi = chi_sqrd_from_groups( '+', '-', classfn, groups.values())
        p_value = stats.chi2.pdf(chi , len(groups) - 1)
        if p_value <= 0.01:
            g, fn, p = gn, f, p_value
            best_groups = groups
#         else:
#             print("rejecting split with p value {0}".format(p_value))


    
    if best_groups:
        print("max gain is {0}: {1}. p value {2}".format(fn.__name__, g, p)) 
        for key, val in best_groups.items():
            print("{0}: {1}".format(key, len(val)))
    return best_groups, fn

In [491]:
x = shuffled_data[:414].values.tolist()

new_root = build_tree(x, cc_split_chi, cc_att_fns, cc_class)

groups in chi_sqrd_from_groups:  [(83, 97), (105, 129)]
groups in chi_sqrd_from_groups:  [(121, 155), (3, 3), (64, 68)]
groups in chi_sqrd_from_groups:  [(4, 2), (173, 197), (11, 27)]
groups in chi_sqrd_from_groups:  [(2, 0), (26, 71), (3, 2), (157, 153)]
groups in chi_sqrd_from_groups:  [(157, 153), (3, 2), (26, 71), (2, 0)]
groups in chi_sqrd_from_groups:  [(5, 30), (99, 134), (56, 27), (4, 1), (13, 21), (2, 3), (1, 1), (3, 4), (2, 1), (3, 4)]
groups in chi_sqrd_from_groups:  [(127, 51), (61, 175)]
groups in chi_sqrd_from_groups:  [(172, 44), (16, 182)]
max gain is eight: 0.41971809924006076. p value 1.2933524229004078e-48
t: 216
f: 198
groups in chi_sqrd_from_groups:  [(109, 32), (2, 0), (61, 12)]
groups in chi_sqrd_from_groups:  [(77, 26), (95, 18)]
groups in chi_sqrd_from_groups:  [(164, 37), (8, 7)]
groups in chi_sqrd_from_groups:  [(148, 30), (24, 14)]
groups in chi_sqrd_from_groups:  [(24, 14), (148, 30)]
groups in chi_sqrd_from_groups:  [(4, 5), (91, 24), (4, 0), (56, 5), (12,

In [492]:
count_nodes(new_root)

11

Ok, let's start classifying examples

In [494]:
def classify(root, sample):
    node = root
    while not isinstance(node, Leaf):
        key = node.predicate(sample)
        node = node.children[key]
    return node.class_label

In [495]:
print(x[0])
classify(new_root, x[0])

['a', '38.33', 4.415, 'u', 'g', 'c', 'v', 0.125, 'f', 'f', 0, 'f', 'g', '00160', 0, '-']


'-'

In [468]:
print(x[140])
classify(new_root, x[140])

['b', '48.17', 3.5, 'u', 'g', 'aa', 'v', 3.5, 't', 'f', 0, 'f', 's', '00230', 0, '+']


'+'

In [472]:
print(x[500])
classify(new_root, x[500])

['b', '18.50', 2.0, 'u', 'g', 'i', 'v', 1.5, 't', 't', 2, 'f', 'g', '00120', 300, '+']


'+'

In [496]:
test_data = shuffled_data[414:].values.tolist()

In [499]:
def accuracy(root, test_data, classfn):
    correct, wrong = 0, 0
    for sample in test_data:
        if classfn(sample) == classify(root, sample):
            correct += 1
        else:
            wrong += 1
    return correct / (correct + wrong)
        

In [500]:
accuracy(new_root, test_data, cc_class)

0.8586956521739131

How does it compare to sklearn?

In [578]:
X = shuffled_data[list(range(15))]
y = shuffled_data[15]

In [540]:
from sklearn.model_selection import cross_val_score
from sklearn import tree
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X, y, cv=5)
print(scores)

ValueError: could not convert string to float: '?'

:( need to handle question marks

In [658]:
print(X[X[0] == '?'].shape)
print(X[X[1] == '?'].shape)
print(X[X[3] == '?'].shape)
print(X[X[4] == '?'].shape)
print(X[X[5] == '?'].shape)
print(X[X[6] == '?'].shape)
print(X[X[8] == '?'].shape)
print(X[X[9] == '?'].shape)
# please, make it stop. Need to figure out better way of doing this
print(X[X[11] == '?'].shape)
print(X[X[12] == '?'].shape)
print(X[X[13] == '?'].shape)


(12, 15)
(12, 15)
(6, 15)
(6, 15)
(9, 15)
(9, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(13, 15)


In [649]:
cd = shuffled_data.copy() #clean data
cd.shape

(690, 16)

In [659]:
for i in [1, 3, 4, 5, 6, 8 , 9, 11, 12, 13]:
    cd = cd[cd[i] != '?'] # cd.drop(cd[i] != '?', inplace=True) didn't work :(

In [652]:
cd.shape

(661, 16)

In [656]:
all(cd[cd[i] == '?'].empty for i in [1, 3, 4, 5, 6, 8 , 9, 11, 12, 13])

True

Ok, we have our clean dataset. Let's try again.

In [693]:
X_clean = pd.get_dummies(cd[list(range(15))]) # boo, won't work without converting categorical data into numbers
y_clean = cd[15]
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X_clean, y_clean, cv=5)
print(scores)

[ 0.86466165  0.84962406  0.82575758  0.84090909  0.85496183]


In [694]:
np.mean(scores)

0.84718284260268995